# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Import" data-toc-modified-id="Data-Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Import</a></div><div class="lev1 toc-item"><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Cleaning</a></div><div class="lev2 toc-item"><a href="#Drop-irrelevant-features" data-toc-modified-id="Drop-irrelevant-features-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Drop irrelevant features</a></div><div class="lev2 toc-item"><a href="#Re-formatted-features" data-toc-modified-id="Re-formatted-features-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Re-formatted features</a></div><div class="lev2 toc-item"><a href="#Missing-Value" data-toc-modified-id="Missing-Value-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Missing Value</a></div><div class="lev2 toc-item"><a href="#Data-Types-in-Pandas" data-toc-modified-id="Data-Types-in-Pandas-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Data Types in Pandas</a></div><div class="lev1 toc-item"><a href="#Categorical-vairable" data-toc-modified-id="Categorical-vairable-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Categorical vairable</a></div><div class="lev2 toc-item"><a href="#Categorical-variable-encoding" data-toc-modified-id="Categorical-variable-encoding-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Categorical variable encoding</a></div><div class="lev3 toc-item"><a href="#Label-encoding" data-toc-modified-id="Label-encoding-311"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Label encoding</a></div><div class="lev3 toc-item"><a href="#One-hot-encoding" data-toc-modified-id="One-hot-encoding-312"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>One-hot encoding</a></div><div class="lev1 toc-item"><a href="#Numerical-variable" data-toc-modified-id="Numerical-variable-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Numerical variable</a></div><div class="lev1 toc-item"><a href="#Output-file" data-toc-modified-id="Output-file-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Output file</a></div>

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import tz
sns.set(color_codes=True)

# Data Import

In [2]:
# load dataset
train_file = 'Dataset/round1_ijcai_18_train_20180301.txt'
test_file = 'Dataset/round1_ijcai_18_test_a_20180301.txt'

df_train = pd.read_csv(train_file, delim_whitespace=True, header=0)
df_test = pd.read_csv(test_file, delim_whitespace=True, header=0)

print("The shape of training_set: ", df_train.shape)
print("The shape of testing_set: ", df_test.shape)

The shape of training_set:  (478138, 27)
The shape of testing_set:  (18371, 26)


* Training set共有47萬筆資料，26個features，1個label
* Test set共有1萬8千筆資料，26個features

In [3]:
df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()
df_train.set_index('instance_id',inplace = True)
df_test.set_index('instance_id',inplace = True)

print("The shape of training_set: ", df_train.shape)
print("The shape of testing_set: ", df_test.shape)

The shape of training_set:  (478111, 26)
The shape of testing_set:  (18371, 25)


# Data Cleaning

1. 將無關的feature捨去
2. 格式化feature
3. 處理missing value
4. Check features data type in Pandas

## Drop irrelevant features
目前考量到`item_property_list`以及`predict_category_property`兩個feature中的值比較難做處理，很難在training時有顯著性，暫時先drop這兩個features。另外，`context_id`和`instance_id`都為唯一id，也將`context_id` drop掉。

In [4]:
df_train = df_train.drop(['item_property_list', 'predict_category_property', 'context_id'], axis=1)
df_test = df_test.drop(['item_property_list', 'predict_category_property', 'context_id'], axis=1)

## Re-formatted features
* item_category_list
* context_timestamp

In [5]:
def formatted_features(df):
    df['cate0'] = df['item_category_list'].apply(lambda x: x.split(';')[0])
    df['cate1'] = df['item_category_list'].apply(lambda x: x.split(';')[1])
    df['item_category_list'] = df['cate1']
    df = df.drop(['cate0', 'cate1'], axis=1)
    df['context_timestamp'] = df['context_timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))
    df['context_timestamp_date'] = df['context_timestamp'].apply(lambda x : x.date()).astype('str')
    df['context_timestamp_hour'] = df['context_timestamp'].apply(lambda x : x.hour)
    df['context_timestamp_weekday'] = df['context_timestamp'].apply(lambda x : x.ctime()).apply(lambda x : x.split(" ")[0])
    df = df.drop(['context_timestamp'], axis=1)
    return df

In [6]:
df_train = formatted_features(df_train)
df_test = formatted_features(df_test)

## Missing Value
原先使用捨棄有NA的樣本，但考量到test set也有NA樣本，所以暫時用NAN處理

In [7]:
df_train[df_train.isin([-1])].count()

item_id                          0
item_category_list               0
item_brand_id                  473
item_city_id                   277
item_price_level                 0
item_sales_level               913
item_collected_level             0
item_pv_level                    0
user_id                          0
user_gender_id               12902
user_age_level                 964
user_occupation_id             964
user_star_level                964
context_page_id                  0
shop_id                          0
shop_review_num_level            0
shop_review_positive_rate        7
shop_star_level                  0
shop_score_service              59
shop_score_delivery             59
shop_score_description          59
is_trade                         0
context_timestamp_date           0
context_timestamp_hour           0
context_timestamp_weekday        0
dtype: int64

In [8]:
df_test[df_test.isin([-1])].count()

item_id                        0
item_category_list             0
item_brand_id                 18
item_city_id                   6
item_price_level               0
item_sales_level              35
item_collected_level           0
item_pv_level                  0
user_id                        0
user_gender_id               469
user_age_level                18
user_occupation_id            18
user_star_level               18
context_page_id                0
shop_id                        0
shop_review_num_level          0
shop_review_positive_rate      0
shop_star_level                0
shop_score_service             1
shop_score_delivery            1
shop_score_description         1
context_timestamp_date         0
context_timestamp_hour         0
context_timestamp_weekday      0
dtype: int64

In [9]:
df_train.user_gender_id.value_counts(normalize=True)

 0    0.754630
 1    0.196739
-1    0.026985
 2    0.021646
Name: user_gender_id, dtype: float64

In [10]:
df_test.user_gender_id.value_counts(normalize=True)

 0    0.749878
 1    0.202711
-1    0.025529
 2    0.021882
Name: user_gender_id, dtype: float64

In [11]:
df_train.replace(-1, np.nan, inplace=True)
df_test.replace(-1, np.nan, inplace=True)

## Data Types in Pandas

In [12]:
for col in list(df_train.columns):
    print(col + ": " + str(df_train[col].dtype))

item_id: int64
item_category_list: object
item_brand_id: float64
item_city_id: float64
item_price_level: int64
item_sales_level: float64
item_collected_level: int64
item_pv_level: int64
user_id: int64
user_gender_id: float64
user_age_level: float64
user_occupation_id: float64
user_star_level: float64
context_page_id: int64
shop_id: int64
shop_review_num_level: int64
shop_review_positive_rate: float64
shop_star_level: int64
shop_score_service: float64
shop_score_delivery: float64
shop_score_description: float64
is_trade: int64
context_timestamp_date: object
context_timestamp_hour: int64
context_timestamp_weekday: object


In [13]:
df_train.item_id = df_train.item_id.astype('int64')
df_train.item_category_list = df_train.item_category_list.astype('int64')
#df_train.item_brand_id = df_train.item_brand_id.astype('int64')
#df_train.item_city_id = df_train.item_city_id.astype('int64')
df_train.item_price_level = df_train.item_price_level.astype('int64')
#df_train.item_sales_level = df_train.item_sales_level.astype('int64')
df_train.item_collected_level = df_train.item_collected_level.astype('int64')
df_train.item_pv_level = df_train.item_pv_level.astype('int64')
df_train.user_id = df_train.user_id.astype('int64')
#df_train.user_gender_id = df_train.user_gender_id.astype('int64')
#df_train.user_age_level = df_train.user_age_level.astype('int64')
#df_train.user_occupation_id = df_train.user_occupation_id.astype('int64')
#df_train.user_star_level = df_train.user_star_level.astype('int64')
df_train.context_page_id = df_train.context_page_id.astype('int64')
df_train.shop_id = df_train.shop_id.astype('int64')
df_train.shop_review_num_level = df_train.shop_review_num_level.astype('int64')
df_train.shop_review_positive_rate = df_train.shop_review_positive_rate.astype('float64')
df_train.shop_star_level = df_train.shop_star_level.astype('int64')
df_train.shop_score_service = df_train.shop_score_service.astype('float64')
df_train.shop_score_delivery = df_train.shop_score_delivery.astype('float64')
df_train.shop_score_description = df_train.shop_score_description.astype('float64')
df_train.is_trade = df_train.is_trade.astype('int64')
df_train.context_timestamp_date = df_train.context_timestamp_date.astype('datetime64[ns]')
df_train.context_timestamp_hour = df_train.context_timestamp_hour.astype('int64')
df_train.context_timestamp_weekday = df_train.context_timestamp_weekday.astype('object')

In [14]:
df_test.item_id = df_test.item_id.astype('int64')
df_test.item_category_list = df_test.item_category_list.astype('int64')
#df_test.item_brand_id = df_test.item_brand_id.astype('int64')
#df_test.item_city_id = df_test.item_city_id.astype('int64')
df_test.item_price_level = df_test.item_price_level.astype('int64')
#df_test.item_sales_level = df_test.item_sales_level.astype('int64')
df_test.item_collected_level = df_test.item_collected_level.astype('int64')
df_test.item_pv_level = df_test.item_pv_level.astype('int64')
df_test.user_id = df_test.user_id.astype('int64')
#df_test.user_gender_id = df_test.user_gender_id.astype('int64')
#df_test.user_age_level = df_test.user_age_level.astype('int64')
#df_test.user_occupation_id = df_test.user_occupation_id.astype('int64')
#df_test.user_star_level = df_test.user_star_level.astype('int64')
df_test.context_page_id = df_test.context_page_id.astype('int64')
df_test.shop_id = df_test.shop_id.astype('int64')
df_test.shop_review_num_level = df_test.shop_review_num_level.astype('int64')
df_test.shop_review_positive_rate = df_test.shop_review_positive_rate.astype('float64')
df_test.shop_star_level = df_test.shop_star_level.astype('int64')
df_test.shop_score_service = df_test.shop_score_service.astype('float64')
df_test.shop_score_delivery = df_test.shop_score_delivery.astype('float64')
df_test.shop_score_description = df_test.shop_score_description.astype('float64')
#df_test.is_trade = df_test.is_trade.astype('int64')
df_test.context_timestamp_date = df_test.context_timestamp_date.astype('datetime64[ns]')
df_test.context_timestamp_hour = df_test.context_timestamp_hour.astype('int64')
df_test.context_timestamp_weekday = df_test.context_timestamp_weekday.astype('object')

# Categorical vairable
有相當多問題要去釐清，由於categorical data在很多機器學習的方法當中需要轉換成One-hot encoding，導致類別多的categorical feature會轉換成sparse matrix。尤其在`id`這個欄位經常發生這個問題。

問題需要澄清：
1. One-hot encoding使用時機
2. Label encoding使用時機
3. 使用sklearn的One-hot encoding其不便性: 難追蹤回原feature
4. On-line分析如何解決training set和test set (online, unknown) categorical不一致問題

共有11個categorical vaiable的feature

In [15]:
fs_cg = ['item_id', 
         'user_id', 
         'shop_id', 
         'item_category_list', 
         'item_brand_id', 
         'item_city_id',  
         'user_gender_id', 
         'user_occupation_id', 
         'context_page_id', 
         'context_timestamp_hour', 
         'context_timestamp_weekday']

fs_cg_ohec = [#'item_id',
              #'shop_id',
              #'user_id',
              'item_category_list',
              #'item_brand_id',
              'item_city_id',
              'user_gender_id',
              'user_occupation_id',
              'context_page_id']

fs_cg_labelec = ['context_timestamp_hour',
                 'context_timestamp_weekday']

fs_cg_drop = ['item_id',
              'shop_id',
              'user_id']

In [16]:
print('Number of unique category (training/test)')
for col in fs_cg:
    print(col + ": " + str(len(df_train[col].unique())) + " / " + str(len(df_test[col].unique())))

Number of unique category (training/test)
item_id: 10075 / 3695
user_id: 197694 / 13573
shop_id: 3959 / 2015
item_category_list: 13 / 13
item_brand_id: 2055 / 1101
item_city_id: 128 / 99
user_gender_id: 4 / 4
user_occupation_id: 5 / 5
context_page_id: 20 / 20
context_timestamp_hour: 24 / 24
context_timestamp_weekday: 7 / 1


## Categorical variable encoding

採用`pandas`的作法而非`sklearn`的做法，方便做feature importance的比較

In [17]:
df_tmp = pd.concat([df_train, df_test], axis=0)

### Label encoding

In [18]:
for col in fs_cg:
    df_tmp[col] = pd.Categorical(df_tmp[col]).codes

### One-hot encoding

In [19]:
df_tmp = pd.get_dummies(df_tmp, columns=fs_cg_ohec)

In [20]:
df_tmp.shape

(496482, 190)

Drop 不處理的 feature

In [21]:
df_tmp = df_tmp.drop(fs_cg_drop, axis=1)

In [22]:
df_train = df_tmp[:df_train.shape[0]]
df_test = df_tmp[df_train.shape[0]:]

In [23]:
print(df_train.shape)
print(df_test.shape)

(478111, 187)
(18371, 187)


# Numerical variable
共有12個numerical variable

In [24]:
fs_num = ['item_price_level', 
          'item_sales_level', 
          'item_collected_level', 
          'item_pv_level', 
          'user_age_level', 
          'user_star_level', 
          'shop_review_num_level', 
          'shop_review_positive_rate', 
          'shop_star_level', 
          'shop_score_service', 
          'shop_score_delivery', 
          'shop_score_description']

fs_num_labelec = ['user_age_level',
                  'user_star_level',
                  'shop_star_level']

In [25]:
df_tmp = pd.concat([df_train, df_test], axis=0)

In [26]:
for col in fs_num_labelec:
    df_tmp[col] = pd.Categorical(df_tmp[col]).codes

In [27]:
df_train = df_tmp[:df_train.shape[0]]
df_test = df_tmp[df_train.shape[0]:]

In [28]:
df_train[fs_num].describe()

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_age_level,user_star_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
count,478111.000000,477198.000000,478111.000000,478111.000000,478111.000000,478111.000000,478111.000000,478104.000000,478111.000000,478052.000000,478052.000000,478052.000000
mean,6.777242,11.156956,12.199288,17.128648,3.470550,4.220208,15.768296,0.994859,14.175187,0.971367,0.970740,0.975107
std,1.088979,2.536313,2.497317,2.160771,1.317455,2.198088,2.825281,0.009223,2.590979,0.009369,0.009281,0.012038
min,0.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.714286,0.000000,0.786667,0.830000,0.786667
25%,6.000000,10.000000,11.000000,16.000000,3.000000,3.000000,14.000000,0.992779,13.000000,0.966363,0.965684,0.969280
50%,7.000000,11.000000,12.000000,17.000000,3.000000,4.000000,16.000000,1.000000,14.000000,0.972347,0.971592,0.978493
75%,8.000000,13.000000,14.000000,19.000000,4.000000,6.000000,17.000000,1.000000,16.000000,0.977822,0.976978,0.983640
max,17.000000,17.000000,17.000000,21.000000,7.000000,10.000000,25.000000,1.000000,21.000000,1.000000,1.000000,1.000000


In [29]:
df_test[fs_num].describe()

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_age_level,user_star_level,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
count,18371.000000,18336.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18371.000000,18370.000000,18370.000000,18370.000000
mean,6.837951,11.132308,12.228621,17.035382,3.464754,4.232432,15.816123,0.994915,14.228948,0.971429,0.970566,0.975148
std,1.046772,2.616461,2.549283,2.237930,1.304611,2.195352,2.879969,0.008981,2.646194,0.009299,0.009288,0.011928
min,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000,1.000000,0.870588,1.000000,0.900728,0.896471,0.858856
25%,6.000000,10.000000,11.000000,16.000000,3.000000,3.000000,14.000000,0.992824,13.000000,0.966399,0.965516,0.969490
50%,7.000000,11.000000,13.000000,17.000000,3.000000,4.000000,16.000000,1.000000,14.000000,0.972481,0.971511,0.978514
75%,8.000000,13.000000,14.000000,19.000000,4.000000,6.000000,18.000000,1.000000,16.000000,0.977937,0.976823,0.983543
max,10.000000,17.000000,17.000000,21.000000,7.000000,10.000000,23.000000,1.000000,21.000000,1.000000,1.000000,1.000000


# Output file

In [30]:
print(df_train.shape)
print(df_test.shape)

(478111, 187)
(18371, 187)


In [31]:
output_train = 'Dataset/training_478111_propdrop_iddrop_onehot.pkl'
output_test = 'Dataset/test_18371_propdrop_iddrop_onehot.pkl'
df_train.to_pickle(output_train)
df_test.to_pickle(output_test)
#df_new_train = pd.read_pickle(output_train)
#df_new_test = pd.read_pickle(output_test)